# Chapter 11

Examples and Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


In [46]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")

In [47]:

import numpy as np
import pandas as pd

import thinkstats2
import thinkplot

## Multiple regression

Let's load up the NSFG data again.

In [48]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/first.py")

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz"
)

In [49]:
import first

live, firsts, others = first.MakeFrames()

Here's birth weight as a function of mother's age (which we saw in the previous chapter).

In [50]:
import statsmodels.formula.api as smf

formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Thu, 28 Jul 2022   Prob (F-statistic):           5.72e-11
Time:                        23:42:43   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.000       6.697       6.964
agepreg        0.0175      0.003      6.559      0.000       0.012       0.023
==============================================================================
Omnibus:                     1024.052   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3081.833
Skew:                          -0.601   Prob(JB):                         0.00
Kurtosis:                       5.596   Cond. No.                         118.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We can extract the parameters.

In [51]:
inter = results.params['Intercept']
slope = results.params['agepreg']
inter, slope

(6.83039697331106, 0.017453851471802652)

And the p-value of the slope estimate.

In [52]:
slope_pvalue = results.pvalues['agepreg']
slope_pvalue

5.722947107316259e-11

And the coefficient of determination.

In [53]:
results.rsquared

0.004738115474710369

The difference in birth weight between first babies and others.

In [54]:
diff_weight = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
diff_weight

-0.12476118453549034

The difference in age between mothers of first babies and others.

In [55]:
diff_age = firsts.agepreg.mean() - others.agepreg.mean()
diff_age

-3.586434766150152

The age difference plausibly explains about half of the difference in weight.

In [56]:
slope * diff_age

-0.06259709972169403

Running a single regression with a categorical variable, `isfirst`:

In [57]:
live['isfirst'] = live.birthord == 1
formula = 'totalwgt_lb ~ isfirst'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     17.74
Date:                Thu, 28 Jul 2022   Prob (F-statistic):           2.55e-05
Time:                        23:42:43   Log-Likelihood:                -15909.
No. Observations:                9038   AIC:                         3.182e+04
Df Residuals:                    9036   BIC:                         3.184e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           7.3259      0.021    356.007      0.000       7.286       7.366
isfirst[T.True]    -0.1248      0.030     -4.212      0.000      -0.183      -0.067
==============================================================================
Omnibus:                      988.919   Durbin-Watson:                   1.613
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2897.107
Skew:                          -0.589   Prob(JB):                         0.00
Kurtosis:                       5.511   Cond. No.                         2.58
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now finally running a multiple regression:

In [58]:
formula = 'totalwgt_lb ~ isfirst + agepreg'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     24.02
Date:                Thu, 28 Jul 2022   Prob (F-statistic):           3.95e-11
Time:                        23:42:43   Log-Likelihood:                -15894.
No. Observations:                9038   AIC:                         3.179e+04
Df Residuals:                    9035   BIC:                         3.182e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           6.9142      0.078     89.073      0.000       6.762       7.066
isfirst[T.True]    -0.0698      0.031     -2.236      0.025      -0.131      -0.009
agepreg             0.0154      0.003      5.499      0.000       0.010       0.021
==============================================================================
Omnibus:                     1019.945   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3063.682
Skew:                          -0.599   Prob(JB):                         0.00
Kurtosis:                       5.588   Cond. No.                         137.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

As expected, when we control for mother's age, the apparent difference due to `isfirst` is cut in half.

If we add age squared, we can control for a quadratic relationship between age and weight.

In [59]:
live['agepreg2'] = live.agepreg**2
formula = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     22.64
Date:                Thu, 28 Jul 2022   Prob (F-statistic):           1.35e-14
Time:                        23:42:43   Log-Likelihood:                -15884.
No. Observations:                9038   AIC:                         3.178e+04
Df Residuals:                    9034   BIC:                         3.181e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.6923      0.286     19.937      0.000       5.133       6.252
isfirst[T.True]    -0.0504      0.031     -1.602      0.109      -0.112       0.011
agepreg             0.1124      0.022      5.113      0.000       0.069       0.155
agepreg2           -0.0018      0.000     -4.447      0.000      -0.003      -0.001
==============================================================================
Omnibus:                     1007.149   Durbin-Watson:                   1.616
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3003.343
Skew:                          -0.594   Prob(JB):                         0.00
Kurtosis:                       5.562   Cond. No.                     1.39e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.39e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

When we do that, the apparent effect of `isfirst` gets even smaller, and is no longer statistically significant.

These results suggest that the apparent difference in weight between first babies and others might be explained by difference in mothers' ages, at least in part.

## Data Mining

We can use `join` to combine variables from the preganancy and respondent tables.

In [60]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dct")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dat.gz")

In [61]:
import nsfg

live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')
join.shape

(8884, 3333)

And we can search for variables with explanatory power.

Because we don't clean most of the variables, we are probably missing some good ones.

In [62]:
import patsy

def GoMining(df):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'totalwgt_lb ~ agepreg + ' + name
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError, patsy.PatsyError) as e:
            continue
        
        variables.append((results.rsquared, name))

    return variables

In [63]:
variables = GoMining(join)
variables

[(0.005357647323640746, 'caseid'),
 (0.005750013985077018, 'pregordr'),
 (0.006330980237390427, 'pregend1'),
 (0.016017752709788224, 'nbrnaliv'),
 (0.005543156193094756, 'cmprgend'),
 (0.005442800591639707, 'cmprgbeg'),
 (0.005327612601560894, 'gestasun_m'),
 (0.007023552638453223, 'gestasun_w'),
 (0.12340041363361054, 'wksgest'),
 (0.027144274639579802, 'mosgest'),
 (0.005336869167517744, 'bpa_bdscheck1'),
 (0.01855092529394209, 'babysex'),
 (0.9498127305978009, 'birthwgt_lb'),
 (0.013102457615706053, 'birthwgt_oz'),
 (0.005543156193094756, 'cmbabdob'),
 (0.005684952650028108, 'kidage'),
 (0.006165319836040184, 'hpagelb'),
 (0.0080663173686768, 'matchfound'),
 (0.012529022541810764, 'anynurse'),
 (0.004409820583625823, 'frsteatd_n'),
 (0.004263973471709592, 'frsteatd_p'),
 (0.004020131462736054, 'frsteatd'),
 (0.005830571770254256, 'cmlastlb'),
 (0.005356747266123563, 'cmfstprg'),
 (0.005428333650990047, 'cmlstprg'),
 (0.005731401733759189, 'cmintstr'),
 (0.005543156193094756, 'cmintf

The following functions report the variables with the highest values of $R^2$.

In [64]:
import re

def ReadVariables():
    """Reads Stata dictionary files for NSFG data.

    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

def MiningReport(variables, n=30):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

Some of the variables that do well are not useful for prediction because they are not known ahead of time.

In [65]:
MiningReport(variables)

totalwgt_lb 1.0
birthwgt_lb 0.9498127305978009 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.3008240784470769 LOW BIRTHWEIGHT - BABY 1
prglngth 0.13012519488625063 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.12340041363361054 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
agecon 0.10203149928156041 AGE AT TIME OF CONCEPTION
mosgest 0.027144274639579802 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
babysex 0.01855092529394209 BD-2 SEX OF 1ST LIVEBORN BABY FROM THIS PREGNANCY
race_r 0.016199503586252995 RACE
race 0.016199503586252995 RACE
nbrnaliv 0.016017752709788224 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
paydu 0.014003795578114597 IB-10 CURRENT LIVING QUARTERS OWNED/RENTED, ETC
rmarout03 0.013430066465713209 INFORMAL MARITAL STATUS WHEN PREGNANCY ENDED - 3RD
birthwgt_oz 0.013102457615706053 BD-3 BIRTHWEIGHT IN OUNCES - 1ST BABY FROM THIS PREGNANCY
anynurse 0.012529022541810764 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM TH

/var/folders/p0/pjkt4x3j3x3fcxxknylhjcq00000gn/T/ipykernel_933/4248865890.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_vars = vars1.append(vars2)


Combining the variables that seem to have the most explanatory power.

In [66]:
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + '
               'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     79.98
Date:                Thu, 28 Jul 2022   Prob (F-statistic):          4.86e-113
Time:                        23:43:08   Log-Likelihood:                -14295.
No. Observations:                8781   AIC:                         2.861e+04
Df Residuals:                    8773   BIC:                         2.866e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                6.6303      0.065    102.223      0.000       6.503       6.757
C(race)[T.2]             0.3570      0.032     11.215      0.000       0.295       0.419
C(race)[T.3]             0.2665      0.051      5.175      0.000       0.166       0.367
babysex == 1[T.True]     0.2952      0.026     11.216      0.000       0.244       0.347
nbrnaliv > 1[T.True]    -1.3783      0.108    -12.771      0.000      -1.590      -1.167
paydu == 1[T.True]       0.1196      0.031      3.861      0.000       0.059       0.180
agepreg                  0.0074      0.003      2.921      0.004       0.002       0.012
totincr                  0.0122      0.004      3.110      0.002       0.005       0.020
==============================================================================
Omnibus:                      398.813   Durbin-Watson:                   1.604
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1388.362
Skew:                          -0.037   Prob(JB):                    3.32e-302
Kurtosis:                       4.947   Cond. No.                         221.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Logistic regression

Example: suppose we are trying to predict `y` using explanatory variables `x1` and `x2`.

In [67]:
y = np.array([0, 1, 0, 1])
x1 = np.array([0, 0, 0, 1])
x2 = np.array([0, 1, 1, 1])

According to the logit model the log odds for the $i$th element of $y$ is

$\log o = \beta_0 + \beta_1 x_1 + \beta_2 x_2 $

So let's start with an arbitrary guess about the elements of $\beta$:



In [68]:
beta = [-1.5, 2.8, 1.1]

Plugging in the model, we get log odds.

In [69]:
log_o = beta[0] + beta[1] * x1 + beta[2] * x2
log_o

array([-1.5, -0.4, -0.4,  2.4])

Which we can convert to odds.

In [70]:
o = np.exp(log_o)
o

array([ 0.22313016,  0.67032005,  0.67032005, 11.02317638])

And then convert to probabilities.

In [71]:
p = o / (o+1)
p

array([0.18242552, 0.40131234, 0.40131234, 0.9168273 ])

The likelihoods of the actual outcomes are $p$ where $y$ is 1 and $1-p$ where $y$ is 0. 

In [72]:
likes = np.where(y, p, 1-p)
likes

array([0.81757448, 0.40131234, 0.59868766, 0.9168273 ])

The likelihood of $y$ given $\beta$ is the product of `likes`:

In [73]:
like = np.prod(likes)
like

0.1800933529673034

Logistic regression works by searching for the values in $\beta$ that maximize `like`.

Here's an example using variables in the NSFG respondent file to predict whether a baby will be a boy or a girl.

In [74]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]
live['boy'] = (live.babysex==1).astype(int)

The mother's age seems to have a small effect.

In [75]:
model = smf.logit('boy ~ agepreg', data=live)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8884
Model:                          Logit   Df Residuals:                     8882
Method:                           MLE   Df Model:                            1
Date:                Thu, 28 Jul 2022   Pseudo R-squ.:               6.144e-06
Time:                        23:43:09   Log-Likelihood:                -6156.7
converged:                       True   LL-Null:                       -6156.8
Covariance Type:            nonrobust   LLR p-value:                    0.7833
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0058      0.098      0.059      0.953      -0.185       0.197
agepreg        0.0010      0.004      0.275      0.783      -0.006       0.009
==============================================================================
"""

Here are the variables that seemed most promising.

In [76]:
formula = 'boy ~ agepreg + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=live)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.692944
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8782
Model:                          Logit   Df Residuals:                     8776
Method:                           MLE   Df Model:                            5
Date:                Thu, 28 Jul 2022   Pseudo R-squ.:               0.0001440
Time:                        23:43:09   Log-Likelihood:                -6085.4
converged:                       True   LL-Null:                       -6086.3
Covariance Type:            nonrobust   LLR p-value:                    0.8822
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.0301      0.104     -0.290      0.772      -0.234       0.173
C(race)[T.2]    -0.0224      0.051     -0.439      0.660      -0.122       0.077
C(race)[T.3]    -0.0005      0.083     -0.005      0.996      -0.163       0.162
agepreg         -0.0027      0.006     -0.484      0.629      -0.014       0.008
hpagelb          0.0047      0.004      1.112      0.266      -0.004       0.013
birthord         0.0050      0.022      0.227      0.821      -0.038       0.048
================================================================================
"""

To make a prediction, we have to extract the exogenous and endogenous variables.

In [77]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

The baseline prediction strategy is to guess "boy".  In that case, we're right almost 51% of the time.

In [78]:
actual = endog['boy']
baseline = actual.mean()
baseline

0.507173764518333

If we use the previous model, we can compute the number of predictions we get right.

In [79]:
predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)
sum(true_pos), sum(true_neg)

(3944.0, 548.0)

And the accuracy, which is slightly higher than the baseline.

In [80]:
acc = (sum(true_pos) + sum(true_neg)) / len(actual)
acc

0.5115007970849464

To make a prediction for an individual, we have to get their information into a `DataFrame`.

In [81]:
columns = ['agepreg', 'hpagelb', 'birthord', 'race']
new = pd.DataFrame([[35, 39, 3, 2]], columns=columns)
y = results.predict(new)
y

0    0.513091
dtype: float64

This person has a 51% chance of having a boy (according to the model).

## Exercises

**Exercise:** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [82]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

The following are the variables I found that have a statistically significant effect on pregnancy length. I am considering these variables as per the analysis.

In [83]:
import statsmodels.formula.api as smf

live, firsts, others = first.MakeFrames()
df=join[join.prglngth>30]
results = smf.ols('prglngth ~ birthord+C(agepreg)+C(diabetes)', data=df).fit()
#print(regression.SummarizeResults(results))
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.062
Date:                Thu, 28 Jul 2022   Prob (F-statistic):              0.211
Time:                        23:43:13   Log-Likelihood:                -18112.
No. Observations:                8884   AIC:                         3.692e+04
Df Residuals:                    8538   BIC:                         3.937e+04
Df Model:                         345                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              38.8133      1.898     20.454      0.000      35.094      42.533
C(agepreg)[T.10.91]  5.624e-14      2.681    2.1e-14      1.000      -5.256       5.256
C(agepreg)[T.11.75]  2.771e-13      2.681   1.03e-13      1.000      -5.256       5.256
C(agepreg)[T.12.66]  1.847e-13      2.681   6.89e-14      1.000      -5.256       5.256
C(agepreg)[T.13.0]     -3.7477      2.682     -1.397      0.162      -9.005       1.510
C(agepreg)[T.13.5]     -3.0000      2.681     -1.119      0.263      -8.256       2.256
C(agepreg)[T.13.58] -2.203e-13      2.681  -8.22e-14      1.000      -5.256       5.256
C(agepreg)[T.13.66]    -1.6448      2.189     -0.751      0.452      -5.936       2.646
C(agepreg)[T.13.75] -4.619e-13      2.681  -1.72e-13      1.000      -5.256       5.256
C(agepreg)[T.13.83] -5.329e-13      2.189  -2.43e-13      1.000      -4.291       4.291
C(agepreg)[T.13.91]     1.3333      2.189      0.609      0.542      -2.958       5.624
C(agepreg)[T.14.0]      3.2523      2.682      1.213      0.225      -2.005       8.510
C(agepreg)[T.14.08] -3.588e-13      2.189  -1.64e-13      1.000      -4.291       4.291
C(agepreg)[T.14.16]    -0.9672      2.322     -0.417      0.677      -5.519       3.584
C(agepreg)[T.14.25]    -2.5000      2.322     -1.077      0.282      -7.051       2.051
C(agepreg)[T.14.33]     1.0000      2.189      0.457      0.648      -3.291       5.291
C(agepreg)[T.14.41]    -4.0000      2.681     -1.492      0.136      -9.256       1.256
C(agepreg)[T.14.5]  -8.171e-14      2.120  -3.86e-14      1.000      -4.155       4.155
C(agepreg)[T.14.58]     0.5000      2.120      0.236      0.814      -3.655       4.655
C(agepreg)[T.14.66]  7.994e-14      2.189   3.65e-14      1.000      -4.291       4.291
C(agepreg)[T.14.75]    -1.2000      2.077     -0.578      0.563      -5.271       2.871
C(agepreg)[T.14.83]     0.1667      2.048      0.081      0.935      -3.847       4.181
C(agepreg)[T.14.91]    -1.0935      2.011     -0.544      0.587      -5.035       2.848
C(agepreg)[T.15.0]     -1.0000      2.027     -0.493      0.622      -4.973       2.973
C(agepreg)[T.15.08]  1.563e-13      2.027   7.71e-14      1.000      -3.973       3.973
C(agepreg)[T.15.16]    -0.4781      2.048     -0.234      0.815      -4.492       3.536
C(agepreg)[T.15.25]    -1.3668      2.011     -0.680      0.497      -5.308       2.575
C(agepreg)[T.15.33]    -0.8000      2.077     -0.385      0.700      -4.871       3.271
C(agepreg)[T.15.41]  1.954e-13      2.011   9.72e-14      1.000      -3.942       3.942
C(agepreg)[T.15.5]     -1.5833      1.973     -0.802      0.422      -5.451       2.285
C(agepreg)[T.15.58]    -1.7498      1.967     -0.889      0.374      -5.606       2.107
C(agepreg)[T.15.66]    -0.7203      1.967     -0.366      0.714      -4.577       3.136
C(a

**Exercise:** The Trivers-Willard hypothesis suggests that for many mammals the sex ratio depends on “maternal condition”; that is, factors like the mother’s age, size, health, and social status. See https://en.wikipedia.org/wiki/Trivers-Willard_hypothesis

Some studies have shown this effect among humans, but results are mixed. In this chapter we tested some variables related to these factors, but didn’t find any with a statistically significant effect on sex ratio.

As an exercise, use a data mining approach to test the other variables in the pregnancy and respondent files. Can you find any factors with a substantial effect?

**Exercise:** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [84]:
def ReadFemResp(dct_file='2002FemResp.dct',
                dat_file='2002FemResp.dat.gz',
                nrows=None):
    """Reads the NSFG respondent data.

    dct_file: string file name
    dat_file: string file name

    returns: DataFrame
    """
    dct = thinkstats2.ReadStataDct(dct_file)
    df = dct.ReadFixedWidth(dat_file, compression='gzip', nrows=nrows)
    return df

def makingJoin():
    live,firsts,others=first.MakeFrames()
    live=live[live.prglngth>30]
    resp=ReadFemResp()
    resp.index=resp.caseid
    join=live.join(resp,on='caseid',rsuffix='_r')
    return join

join=makingJoin()

In [85]:
join=join.dropna(subset=['agepreg'])
join.numbabes.replace([97],np.nan,inplace=True)
join['is35']=((join.agepreg>=35)&(join.agepreg<36)).astype(int)
formula='numbabes~+C(race)+age_r+C(havedeg)+C(totinc)'
model=smf.poisson(formula,data=join)
results=model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.586266
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 4239
Model:                        Poisson   Df Residuals:                     4218
Method:                           MLE   Df Model:                           20
Date:                Thu, 28 Jul 2022   Pseudo R-squ.:                 0.01613
Time:                        23:43:25   Log-Likelihood:                -6724.2
converged:                       True   LL-Null:                       -6834.4
Covariance Type:            nonrobust   LLR p-value:                 9.888e-36
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1345      0.100      1.345      0.179      -0.062       0.331
C(race)[T.2]         -0.0509      0.024     -2.082      0.037      -0.099      -0.003
C(race)[T.3]         -0.0722      0.041     -1.747      0.081      -0.153       0.009
C(havedeg)[T.5.0]     0.0497      0.021      2.334      0.020       0.008       0.091
C(totinc)[T.2]        0.0624      0.102      0.610      0.542      -0.138       0.263
C(totinc)[T.3]        0.2575      0.096      2.671      0.008       0.069       0.447
C(totinc)[T.4]        0.3055      0.090      3.389      0.001       0.129       0.482
C(totinc)[T.5]        0.1083      0.096      1.122      0.262      -0.081       0.297
C(totinc)[T.6]        0.1836      0.086      2.131      0.033       0.015       0.352
C(totinc)[T.7]        0.0953      0.087      1.098      0.272      -0.075       0.266
C(totinc)[T.8]        0.0441      0.087      0.508      0.612      -0.126       0.215
C(totinc)[T.9]        0.0146      0.088      0.166      0.868      -0.157       0.186
C(totinc)[T.10]       0.0197      0.088      0.223      0.824      -0.154       0.193
C(totinc)[T.11]      -0.0378      0.086     -0.440      0.660      -0.206       0.131
C(totinc)[T.12]      -0.0291      0.085     -0.341      0.733      -0.196       0.138
C(totinc)[T.13]      -0.0168      0.085     -0.197      0.844      -0.184       0.150
C(totinc)[T.14]      -0.0394      0.082     -0.481      0.630      -0.200       0.121
C(totinc)[T.97]       0.1448      0.288      0.502      0.616      -0.420       0.710
C(totinc)[T.98]      -0.0501      0.099     -0.507      0.612      -0.244       0.144
C(totinc)[T.99]       0.0711      0.131      0.545      0.586      -0.185       0.327
age_r                 0.0208      0.002     12.038      0.000       0.017       0.024
=====================================================================================
"""

Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000

In [86]:
# Solution goes here
columns = ['race', 'age_r', 'havedeg', 'totinc']
new = pd.DataFrame([[1, 35, 1, 14]], columns=columns)
y = results.predict(new)
y

0    2.279438
dtype: float64

**Exercise:** If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [87]:
# Solution goes here
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

resp=ReadFemResp()
formula='rmarital ~ age_r +C(race)+C(educat)+C(totincr)'
model=smf.mnlogit(formula,data=resp)
results=model.fit()
results.summary()

         Current function value: 1.054219
         Iterations: 35


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 7643
Model:                        MNLogit   Df Residuals:                     7508
Method:                           MLE   Df Model:                          130
Date:                Thu, 28 Jul 2022   Pseudo R-squ.:                  0.1906
Time:                        23:43:36   Log-Likelihood:                -8057.4
converged:                      False   LL-Null:                       -9954.7
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
      rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            2.5873      0.319      8.102      0.000       1.961       3.213
C(race)[T.2]        -0.6489      0.112     -5.790      0.000      -0.869      -0.429
C(race)[T.3]        -0.7345      0.173     -4.248      0.000      -1.073      -0.396
C(educat)[T.10]      0.3770      0.197      1.915      0.055      -0.009       0.763
C(educat)[T.11]     -0.2196      0.210     -1.044      0.296      -0.632       0.193
C(educat)[T.12]     -0.1215      0.138     -0.881      0.378      -0.392       0.149
C(educat)[T.13]     -0.2167      0.172     -1.262      0.207      -0.553       0.120
C(educat)[T.14]     -0.0890      0.168     -0.529      0.596      -0.419       0.241
C(educat)[T.15]     -0.3148      0.214     -1.469      0.142      -0.735       0.105
C(educat)[T.16]     -0.7445      0.195     -3.812      0.000      -1.127      -0.362
C(educat)[T.17]     -0.5642      0.273     -2.068      0.039      -1.099      -0.030
C(educat)[T.18]     -1.1754      0.409     -2.872      0.004      -1.977      -0.373
C(educat)[T.19]      0.1120      0.288      0.389      0.697      -0.452       0.676
C(totincr)[T.2]      0.5144      0.331      1.553      0.121      -0.135       1.164
C(totincr)[T.3]      0.1893      0.341      0.555      0.579      -0.480       0.858
C(totincr)[T.4]     -0.1125      0.308     -0.365      0.715      -0.716       0.491
C(totincr)[T.5]     -0.1790      0.287     -0.624      0.533      -0.742       0.383
C(totincr)[T.6]     -0.3620      0.281     -1.290      0.197      -0.912       0.188
C(totincr)[T.7]     -0.3301      0.279     -1.184      0.236      -0.876       0.216
C(totincr)[T.8]     -0.3530      0.276     -1.279      0.201      -0.894       0.188
C(totincr)[T.9]     -0.5145      0.280     -1.837      0.066      -1.064       0.034
C(totincr)[T.10]    -0.4745      0.290     -1.635      0.102      -1.043       0.094
C(totincr)[T.11]    -0.8088      0.282     -2.873      0.004      -1.361      -0.257
C(totincr)[T.12]    -1.1122      0.288     -3.860      0.000      -1.677      -0.547
C(totincr)[T.13]    -1.1297      0.294     -3.838      0.000      -1.707      -0.553
C(totincr)[T.14]    -1.3146      0.279     -4.711      0.000      -1.862      -0.768
age_r               -0.0864      0.006    -14.865      0.000      -0.098      -0.075
------------------------------------------------------------------------------------
      rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -7.4010      1.299     -5.697      0.000      -9.947      -4.855
C(race)[T.2]        -0.1220      0.366     -0.333      0.739      -0.840       0.596
C(race)[T.3]         0.4449      0.514      0.865      0.387      -0.563       1.453
C(educat)[T.10]      0.3587      0.699      0.513      0.608      -1.011       1.729
C(educat)[T.11]      1.1767      0.514      2.288      0.022       0.169       2.185
C(educat)[T

Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [88]:
# Solution goes here
columns=['age_r','race','educat','totincr']
new=pd.DataFrame([[25,2,12,11]],columns=columns)
y=results.predict(new)
y

,0,1,2,3,4,5
0,0.434947,0.137383,0.000447,0.026985,0.017036,0.383201
